In [2]:
import PyPDF2
import regex as re
from nltk import word_tokenize 
from nltk.util import ngrams
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from dictionary import *
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import json
import os
import glob
import pandas as pd
# from tqdm import tqdm
# filename is test.pdf

In [2]:
os.chdir('/home/uttkarsh/IP/')
print(os.getcwd())
def get_text(file):
    text = ''
    with open(file, 'rb') as pdfFileObj:
        pdfReader = PyPDF2.PdfReader(pdfFileObj)
        num_pages = len(pdfReader.pages)
        for i in range(num_pages):
            pageObj = pdfReader.pages[i]
            text += pageObj.extract_text()
        text = text.replace('\n', '')
    return text

def get_string_from_list(list):
    string = ''
    for i in range(len(list)):
        string += list[i]
        if i != len(list)-1:
            string += ' '
    return string

def filter(text):
    token = word_tokenize(text)
    token = [word.lower() for word in token]
    token = [word for word in token if word.isalpha()]
    token = [word for word in token if (word == 'down' or not word in stop_words )]
    # remoev single character
    token = [word for word in token if len(word) > 1]
    # remove th
    token = [word for word in token if word != 'th']
    for i in range(len(token)):
        if token[i] == 'per' and token[i+1] == 'cent':
            token[i] = 'percent'
            token[i+1] = ''
    token = [word for word in token if word != '']
    return token

def get_bigram(token):
    token = [stemmer.stem(word) for word in token]
    bigram = ngrams(token, 2)
    return list(bigram)

def get_trigram(token):
    token = [stemmer.stem(word) for word in token]
    trigram = ngrams(token, 3)
    return list(trigram)

/home/uttkarsh/IP


In [3]:
def searchAndCount_single(bigram_list, term, hdlist):
    term = stemmer.stem(term)
    count = 0
    for i in range(len(bigram_list)):
        if term in bigram_list[i]:
            for j in range(len(hdlist)):
                if i-7 < 0:
                    for k in range(i+7):
                        if hdlist[j] in bigram_list[k]:
                            count += 1
                elif i+7 > len(bigram_list):
                    for k in range(i, len(bigram_list)):
                        if hdlist[j] in bigram_list[k]:
                            count += 1
                else:
                    for k in range(i-7, i+7):
                        if hdlist[j] in bigram_list[k]:
                            count += 1
    return count


def searchAndCount_double(bigram_list, term1, term2, hdlist):
    term1 = stemmer.stem(term1)
    term2 = stemmer.stem(term2)

    count = 0
    for i in range(len(bigram_list)):
        if term1 in bigram_list[i] and term2 in bigram_list[i]:
            for j in range(len(hdlist)):
                if i-7 < 0:
                    for k in range(i+7):
                        if hdlist[j] in bigram_list[k]:
                            count += 1
                elif i+7 > len(bigram_list):
                    for k in range(i, len(bigram_list)):
                        if hdlist[j] in bigram_list[k]:
                            count += 1
                else:
                    for k in range(i-7, i+7):
                        if hdlist[j] in bigram_list[k]:
                            count += 1
    return count

def searchAndCount_triple(trigram_list, term1, term2, term3, hdlist):
    term1 = stemmer.stem(term1)
    term2 = stemmer.stem(term2)
    term3 = stemmer.stem(term3)
    count = 0
    for i in range(len(trigram_list)):
        if term1 in trigram_list[i] and term2 in trigram_list[i] and term3 in trigram_list[i]:
            for j in range(len(hdlist)):
                if i-7 < 0:
                    for k in range(i+7):
                        if hdlist[j] in trigram_list[k]:
                            count += 1
                elif i+7 > len(trigram_list):
                    for k in range(i, len(trigram_list)):
                        if hdlist[j] in trigram_list[k]:
                            count += 1
                else:
                    for k in range(i-7, i+7):
                        if hdlist[j] in trigram_list[k]:
                            count += 1
    return count


In [4]:
# there is  a file named 'Governer Speeches' which contain all the folders with speeches in them. I want to open the files on by one and run the code on them.
# after running the code I want to make dictionary of the net hawkishness of each speech and the date of the speech. and save it as a json file with the same name as the file name i run the code
# go to the folder 'Governer Speeches' and for all the subfolders in it open eaach pdf file in it and run the code

# opening each folder
os.chdir('C:/Users/karti/OneDrive/Desktop/IP/IP')

def get_folder_names():
    cwd = os.getcwd()
    print(cwd)
    folder_names = glob.glob(cwd + '/Governor Speeches/*')
    return folder_names

# opening each file in each folder
def make_wordcloud_image():
    folder_names = get_folder_names()
    print(folder_names)
    # show this as tqdm
    for i in range(len(folder_names)):
        print(folder_names[i])
        os.chdir(folder_names[i])
        file_names = glob.glob('*.pdf')
        for file_name in file_names:
            print(file_name)
            text = get_text(file_name)
            file_name = file_name[:-4] # removing .pdf from the file name
            if os.path.exists(file_name + '_trigram' + '.json'):
                os.remove(file_name + '_trigram' + '.json')
            if os.path.exists(file_name + '_bigram' + '.json'):
                os.remove(file_name + '_bigram' + '.json')
            token = filter(text)
            N = len(token)
            string_list = get_string_from_list(token)
            bigram_list = get_bigram(token)
            trigram_list = get_trigram(token)
            wordcloud = WordCloud(width=1600, height=800, max_font_size=200).generate(string_list)
            wordcloud.to_file(file_name + '.png')

            cpi_hawkish = searchAndCount_single(bigram_list, 'cpi', ConsumerPricesInflation_Hawkish) + searchAndCount_double(bigram_list, 'cpi', 'inflation', ConsumerPricesInflation_Hawkish) + searchAndCount_triple(trigram_list, 'consumer', 'prices', 'inflation', ConsumerPricesInflation_Hawkish) + searchAndCount_single(bigram_list, 'inflation', ConsumerPricesInflation_Hawkish)
            cpi_dovish = searchAndCount_single(bigram_list, 'cpi', ConsumerPricesInflation_Dovish) + searchAndCount_double(bigram_list, 'cpi', 'inflation', ConsumerPricesInflation_Dovish) + searchAndCount_triple(trigram_list, 'consumer', 'prices', 'inflation', ConsumerPricesInflation_Dovish) + searchAndCount_single(bigram_list, 'inflation', ConsumerPricesInflation_Dovish)
            # --------------------------------------------------------------------------------------------
            coreinflation_hawkish = searchAndCount_double(bigram_list, 'core', 'inflation', ConsumerPricesInflation_Hawkish)
            coreinflation_dovish = searchAndCount_double(bigram_list, 'core', 'inflation', ConsumerPricesInflation_Dovish)
            # --------------------------------------------------------------------------------------------
            ip_inflation_hawkish = searchAndCount_single(bigram_list, 'inflation', InflationPressure_Hawkish)
            ip_inflation_dovish = searchAndCount_single(bigram_list, 'inflation', InflationPressure_Dovish)
            # --------------------------------------------------------------------------------------------
            cs_hawkish = searchAndCount_double(bigram_list, 'consumer', 'spending', ConsumerSpending_Hawkish)
            cs_dovish = searchAndCount_double(bigram_list, 'consumer', 'spending', ConsumerSpending_Dovish)
            # --------------------------------------------------------------------------------------------
            e_hawkish = searchAndCount_single(bigram_list, 'economic', EconomicActivity_Hawkish)
            e_dovish = searchAndCount_single(bigram_list, 'economic', EconomicActivity_Dovish)
            # --------------------------------------------------------------------------------------------
            ea_hawkish = searchAndCount_double(bigram_list, 'economic', 'activity', EconomicActivity_Hawkish)
            ea_dovish = searchAndCount_double(bigram_list, 'economic', 'activity', EconomicActivity_Dovish)
            # --------------------------------------------------------------------------------------------
            ru_hawkish = searchAndCount_double(bigram_list, 'resource', 'utilisation', ResourceUtilization_Hawkish) + searchAndCount_double(bigram_list, 'resource', 'utilization', ResourceUtilization_Hawkish)
            ru_dovish = searchAndCount_double(bigram_list, 'resource', 'utilisation', ResourceUtilization_Dovish) + searchAndCount_double(bigram_list, 'resource', 'utilization', ResourceUtilization_Dovish)
            # --------------------------------------------------------------------------------------------
            em_hawkish = searchAndCount_single(bigram_list, 'employment', Employment_Hawkish)
            em_dovish = searchAndCount_single(bigram_list, 'employment', Employment_Dovish)
            # --------------------------------------------------------------------------------------------
            lm_hawkish = searchAndCount_single(bigram_list, 'labour', LaborMarket_Hawkish) + searchAndCount_double(bigram_list, 'labour', 'market', LaborMarket_Hawkish)
            lm_dovish = searchAndCount_single(bigram_list, 'labour', LaborMarket_Dovish) + searchAndCount_double(bigram_list, 'labour', 'market', LaborMarket_Dovish)
            # --------------------------------------------------------------------------------------------
            uem_hawkish = searchAndCount_single(bigram_list, 'unemployment', Unemployment_Hawkish)
            uem_dovish = searchAndCount_single(bigram_list, 'unemployment', Unemployment_Dovish)

            # make a dictionary of all above values and dump it in a json file
            dict = {'cpi_hawkish': cpi_hawkish, 'cpi_dovish': cpi_dovish, 'coreinflation_hawkish': coreinflation_hawkish, 'coreinflation_dovish': coreinflation_dovish, 'ip_inflation_hawkish': ip_inflation_hawkish, 'ip_inflation_dovish': ip_inflation_dovish, 'cs_hawkish': cs_hawkish, 'cs_dovish': cs_dovish, 'e_hawkish': e_hawkish, 'e_dovish': e_dovish, 'ea_hawkish': ea_hawkish, 'ea_dovish': ea_dovish, 'ru_hawkish': ru_hawkish, 'ru_dovish': ru_dovish, 'em_hawkish': em_hawkish, 'em_dovish': em_dovish, 'lm_hawkish': lm_hawkish, 'lm_dovish': lm_dovish, 'uem_hawkish': uem_hawkish, 'uem_dovish': uem_dovish}
            total_hawk = cpi_hawkish + coreinflation_hawkish + ip_inflation_hawkish + cs_hawkish + e_hawkish + ea_hawkish + ru_hawkish + em_hawkish + lm_hawkish + uem_hawkish
            total_dovish = cpi_dovish + coreinflation_dovish + ip_inflation_dovish + cs_dovish + e_dovish + ea_dovish + ru_dovish + em_dovish + lm_dovish + uem_dovish
            if(total_hawk + total_dovish == 0):
                if(os.path.exists(file_name + '.pdf')):
                    os.remove(file_name + '.pdf')
                if(os.path.exists(file_name + '.png')):
                    os.remove(file_name + '.png')
                print('bye bye bitch')
                continue
            apel_net_sentiment =1 + (total_hawk - total_dovish)/(total_hawk + total_dovish)
            new_net_sentiment = 100*(total_hawk - total_dovish)/(.5*N)
            dict['apel_net_sentiment'] = apel_net_sentiment # type: ignore
            dict['new_net_sentiment'] = new_net_sentiment
            with open(file_name + '.json', 'w') as fp:
                json.dump(dict, fp, indent=4)
            
    return None

In [5]:
make_wordcloud_image()

/home/uttkarsh/IP
['/home/uttkarsh/IP/Governor Speeches/2004 Q2', '/home/uttkarsh/IP/Governor Speeches/2005 Q1', '/home/uttkarsh/IP/Governor Speeches/2008 Q3', '/home/uttkarsh/IP/Governor Speeches/2021 Q1', '/home/uttkarsh/IP/Governor Speeches/2018 Q1', '/home/uttkarsh/IP/Governor Speeches/2014 Q1', '/home/uttkarsh/IP/Governor Speeches/2019 Q3', '/home/uttkarsh/IP/Governor Speeches/2014 Q2', '/home/uttkarsh/IP/Governor Speeches/2017 Q4', '/home/uttkarsh/IP/Governor Speeches/2007 Q1', '/home/uttkarsh/IP/Governor Speeches/2011 Q3', '/home/uttkarsh/IP/Governor Speeches/2005 Q3', '/home/uttkarsh/IP/Governor Speeches/2009 Q4', '/home/uttkarsh/IP/Governor Speeches/2023 Q2', '/home/uttkarsh/IP/Governor Speeches/2011 Q2', '/home/uttkarsh/IP/Governor Speeches/2022 Q3', '/home/uttkarsh/IP/Governor Speeches/2011 Q1', '/home/uttkarsh/IP/Governor Speeches/2016 Q1', '/home/uttkarsh/IP/Governor Speeches/2023 Q1', '/home/uttkarsh/IP/Governor Speeches/2009 Q3', '/home/uttkarsh/IP/Governor Speeches/2016

/home/uttkarsh/IP/Governor Speeches/2005 Q1
12 Feb 05 MP.pdf
08 Jan 05 MP.pdf
/home/uttkarsh/IP/Governor Speeches/2008 Q3
4 July 08 MP.pdf
/home/uttkarsh/IP/Governor Speeches/2021 Q1
/home/uttkarsh/IP/Governor Speeches/2018 Q1
/home/uttkarsh/IP/Governor Speeches/2014 Q1
26 Feb 14 MP.pdf
/home/uttkarsh/IP/Governor Speeches/2019 Q3
/home/uttkarsh/IP/Governor Speeches/2014 Q2
10 Apr 14 MP.pdf
/home/uttkarsh/IP/Governor Speeches/2017 Q4
27 Oct 17 MP.pdf
16 Nov 17 MP.pdf
20 Nov 17 MP.pdf
/home/uttkarsh/IP/Governor Speeches/2007 Q1
06 Mar 07 FM.pdf
/home/uttkarsh/IP/Governor Speeches/2011 Q3
27 Sep 11 MP.pdf
/home/uttkarsh/IP/Governor Speeches/2005 Q3
3 Sep 05 MP.pdf
/home/uttkarsh/IP/Governor Speeches/2009 Q4
5 Oct 09 MP.pdf
16 Oct 09 MP.pdf
/home/uttkarsh/IP/Governor Speeches/2023 Q2
27 Apr 23 FM.pdf
/home/uttkarsh/IP/Governor Speeches/2011 Q2
17 Apr 11 FM.pdf
23 June 11 MP.pdf
10 May 11 MP.pdf
/home/uttkarsh/IP/Governor Speeches/2022 Q3
9 July 22 MP.pdf
5 Sep 22 FM.pdf
/home/uttkarsh/IP/G

In [ ]:
surprise_hd = {}
def fn(df):
    for date in df['Meeting_Date']:
        dates = date.split('/')
        file = dates[0] + '-' + dates[2] + '.pdf'
        text = get_text(file)
        token = filter(text)
        bigram_list = get_bigram(token)
        trigram_list = get_trigram(token)
        N = len(token)
        string_list = get_string_from_list(token)
        cpi_hawkish = searchAndCount_single(bigram_list, 'cpi', ConsumerPricesInflation_Hawkish) + searchAndCount_double(bigram_list, 'cpi', 'inflation', ConsumerPricesInflation_Hawkish) + searchAndCount_triple(trigram_list, 'consumer', 'prices', 'inflation', ConsumerPricesInflation_Hawkish) + searchAndCount_single(bigram_list, 'inflation', ConsumerPricesInflation_Hawkish)
        cpi_dovish = searchAndCount_single(bigram_list, 'cpi', ConsumerPricesInflation_Dovish) + searchAndCount_double(bigram_list, 'cpi', 'inflation', ConsumerPricesInflation_Dovish) + searchAndCount_triple(trigram_list, 'consumer', 'prices', 'inflation', ConsumerPricesInflation_Dovish) + searchAndCount_single(bigram_list, 'inflation', ConsumerPricesInflation_Dovish)
        # --------------------------------------------------------------------------------------------
        coreinflation_hawkish = searchAndCount_double(bigram_list, 'core', 'inflation', ConsumerPricesInflation_Hawkish)
        coreinflation_dovish = searchAndCount_double(bigram_list, 'core', 'inflation', ConsumerPricesInflation_Dovish)
        # --------------------------------------------------------------------------------------------
        ip_inflation_hawkish = searchAndCount_double(bigram_list, 'inflation', 'pressure', InflationPressure_Hawkish)
        ip_inflation_dovish = searchAndCount_double(bigram_list, 'inflation', 'pressure', InflationPressure_Dovish)
        # --------------------------------------------------------------------------------------------
        cs_hawkish = searchAndCount_double(bigram_list, 'consumer', 'spending', ConsumerSpending_Hawkish)
        cs_dovish = searchAndCount_double(bigram_list, 'consumer', 'spending', ConsumerSpending_Dovish)
        # --------------------------------------------------------------------------------------------
        e_hawkish = searchAndCount_single(bigram_list, 'economic', EconomicActivity_Hawkish)
        e_dovish = searchAndCount_single(bigram_list, 'economic', EconomicActivity_Dovish)
        # --------------------------------------------------------------------------------------------
        ea_hawkish = searchAndCount_double(bigram_list, 'economic', 'activity', EconomicActivity_Hawkish)
        ea_dovish = searchAndCount_double(bigram_list, 'economic', 'activity', EconomicActivity_Dovish)
        # --------------------------------------------------------------------------------------------
        ru_hawkish = searchAndCount_double(bigram_list, 'resource', 'utilisation', ResourceUtilization_Hawkish) + searchAndCount_double(bigram_list, 'resource', 'utilization', ResourceUtilization_Hawkish)
        ru_dovish = searchAndCount_double(bigram_list, 'resource', 'utilisation', ResourceUtilization_Dovish) + searchAndCount_double(bigram_list, 'resource', 'utilization', ResourceUtilization_Dovish)
        # --------------------------------------------------------------------------------------------
        em_hawkish = searchAndCount_single(bigram_list, 'employment', Employment_Hawkish)
        em_dovish = searchAndCount_single(bigram_list, 'employment', Employment_Dovish)
        # --------------------------------------------------------------------------------------------
        lm_hawkish = searchAndCount_single(bigram_list, 'labour', LaborMarket_Hawkish) + searchAndCount_double(bigram_list, 'labour', 'market', LaborMarket_Hawkish)
        lm_dovish = searchAndCount_single(bigram_list, 'labour', LaborMarket_Dovish) + searchAndCount_double(bigram_list, 'labour', 'market', LaborMarket_Dovish)
        # --------------------------------------------------------------------------------------------
        uem_hawkish = searchAndCount_single(bigram_list, 'unemployment', Unemployment_Hawkish)
        uem_dovish = searchAndCount_single(bigram_list, 'unemployment', Unemployment_Dovish)

        # make a dictionary of all above values and dump it in a json file
        dict = {'cpi_hawkish': cpi_hawkish, 'cpi_dovish': cpi_dovish, 'coreinflation_hawkish': coreinflation_hawkish, 'coreinflation_dovish': coreinflation_dovish, 'ip_inflation_hawkish': ip_inflation_hawkish, 'ip_inflation_dovish': ip_inflation_dovish, 'cs_hawkish': cs_hawkish, 'cs_dovish': cs_dovish, 'e_hawkish': e_hawkish, 'e_dovish': e_dovish, 'ea_hawkish': ea_hawkish, 'ea_dovish': ea_dovish, 'ru_hawkish': ru_hawkish, 'ru_dovish': ru_dovish, 'em_hawkish': em_hawkish, 'em_dovish': em_dovish, 'lm_hawkish': lm_hawkish, 'lm_dovish': lm_dovish, 'uem_hawkish': uem_hawkish, 'uem_dovish': uem_dovish}
        total_hawk = cpi_hawkish + coreinflation_hawkish + ip_inflation_hawkish + cs_hawkish + e_hawkish + ea_hawkish + ru_hawkish + em_hawkish + lm_hawkish + uem_hawkish
        total_dovish = cpi_dovish + coreinflation_dovish + ip_inflation_dovish + cs_dovish + e_dovish + ea_dovish + ru_dovish + em_dovish + lm_dovish + uem_dovish
        if(total_hawk + total_dovish == 0):
            print('bhai is file ne to kaat dia')
            continue
        apel_net_sentiment =1 + (total_hawk - total_dovish)/(total_hawk + total_dovish)
        # new_net_sentiment = 100*(total_hawk - total_dovish)/(.5*N)
        surprise_hd[date] = {}
        surprise_hd[date]['hd'] = (apel_net_sentiment)
        surprise_hd[date]['surprise'] = (df[df['Meeting_Date'] == date]['MP_Surprise'].values[0])
    return None
        

In [32]:
# access each json file in each folder of the Governor Speeches folder

surprise_hd_temp = {}
os.chdir('C:/Users/karti/OneDrive/Desktop/IP/IP')

def get_folder_names():
    cwd = os.getcwd()
    print(cwd)
    folder_names = glob.glob(cwd + '/Governor Speeches/*')
    return folder_names

folder_names = get_folder_names()
print(len(folder_names))

for i in range(len(folder_names)):
    os.chdir(folder_names[i])
    print(folder_names[i])
    file_names = glob.glob('*.json')
    for file_name in file_names:
        print(file_name)
        # open each json file and get the date and the net sentiment and store them in a csv file
        with open(file_name) as f:
            data = json.load(f)
        date = file_name[:-5]
        surprise_hd_temp[date] = {}
        surprise_hd_temp[date]['apel_net_sentiment'] = data['apel_net_sentiment'] 
        surprise_hd_temp[date]['new_net_sentiment'] = data['new_net_sentiment']
print(surprise_hd_temp)

df = pd.DataFrame.from_dict(surprise_hd_temp, orient='index')
df.to_csv('hd_data.csv')
print(df.head())



C:\Users\karti\OneDrive\Desktop\IP\IP
53
C:\Users\karti\OneDrive\Desktop\IP\IP/Governor Speeches\2004 Q1
C:\Users\karti\OneDrive\Desktop\IP\IP/Governor Speeches\2004 Q2
24 Apr 04 MP.json
C:\Users\karti\OneDrive\Desktop\IP\IP/Governor Speeches\2004 Q4
C:\Users\karti\OneDrive\Desktop\IP\IP/Governor Speeches\2005 Q1
08 Jan 05 MP.json
12 Feb 05 MP.json
C:\Users\karti\OneDrive\Desktop\IP\IP/Governor Speeches\2005 Q3
3 Sep 05 MP.json
C:\Users\karti\OneDrive\Desktop\IP\IP/Governor Speeches\2005 Q4
4 Nov 05 MP.json
C:\Users\karti\OneDrive\Desktop\IP\IP/Governor Speeches\2006 Q1
C:\Users\karti\OneDrive\Desktop\IP\IP/Governor Speeches\2006 Q2
C:\Users\karti\OneDrive\Desktop\IP\IP/Governor Speeches\2006 Q3
C:\Users\karti\OneDrive\Desktop\IP\IP/Governor Speeches\2006 Q4
C:\Users\karti\OneDrive\Desktop\IP\IP/Governor Speeches\2007 Q1
06 Mar 07 FM.json
C:\Users\karti\OneDrive\Desktop\IP\IP/Governor Speeches\2007 Q2
28 May 07 MP.json
9 Apr 07 FM.json
C:\Users\karti\OneDrive\Desktop\IP\IP/Governor Spe